Import libraries

In [1]:

import pandas as pd 
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium

In [2]:
# geolocator = Nominatim(user_agent="MyLocator")
# indirizzo = "Corso Italia, Genova"
# location = geolocator.geocode(indirizzo)

# if location:
#     print(f"indirizzo: {location.address}")
#     print(f"latitudine: {location.latitude}, longitudine: {location.longitude}")
# else:
#     print("Indirizzo non trovato")

General func:\
it returns a dict with information about the address given in input (or list of address)

In [3]:
def verifica_indirizzi(lista_indirizzi):
    geolocator = Nominatim(user_agent='MyAPP')
    risultati = {}
    for ind in lista_indirizzi:
        try :
            location = geolocator.geocode(ind, timeout = 30, addressdetails=True)
            print(location)
            if location : 
                dettagli = location.raw.get('address',{})
                print(f"Dettagli indirizzo per '{ind}': {dettagli}")
                # citta
                citta = dettagli.get('city') or dettagli.get('village') or ('Non disponibile')
                # regione
                regione = dettagli.get('state', 'Non disponibile')
                # nazione
                stato = dettagli.get('country', 'Non disponibile')
                

                risultati[ind] = (True, location.latitude, location.longitude, citta, regione, stato)
            else:
                risultati[ind] =(False, None, None, None, None, None)
        except GeocoderTimedOut:
            risultati[ind] = (False, None, None, 'Timeout', 'Non disponibile', 'Non disponibile')
        except Exception as e:
            risultati[ind] = (False, None, None, f'Errore: {e}', 'Non disponibile', 'Non disponibile')
    
    return risultati


In [4]:
list_indi = ['Viale Italia 17', 'Località Moggie, Zignago']


In [5]:
output = verifica_indirizzi(list_indi)

17, Viale Italia, Rimini, Emilia-Romagna, 47921, Italia
Dettagli indirizzo per 'Viale Italia 17': {'house_number': '17', 'road': 'Viale Italia', 'city': 'Rimini', 'county': 'Rimini', 'ISO3166-2-lvl6': 'IT-RN', 'state': 'Emilia-Romagna', 'ISO3166-2-lvl4': 'IT-45', 'postcode': '47921', 'country': 'Italia', 'country_code': 'it'}
Località Moggie, Pieve, Zignago, La Spezia, Liguria, Italia
Dettagli indirizzo per 'Località Moggie, Zignago': {'road': 'Località Moggie', 'village': 'Pieve', 'municipality': 'Zignago', 'county': 'La Spezia', 'ISO3166-2-lvl6': 'IT-SP', 'state': 'Liguria', 'ISO3166-2-lvl4': 'IT-42', 'country': 'Italia', 'country_code': 'it'}


In [6]:
output

{'Viale Italia 17': (True,
  44.066877,
  12.5373394,
  'Rimini',
  'Emilia-Romagna',
  'Italia'),
 'Località Moggie, Zignago': (True,
  44.2756399,
  9.7367293,
  'Pieve',
  'Liguria',
  'Italia')}

general function: 
it generates an html map where the addresses are marked based on coordinates obtained previously

In [7]:
def mappa_dinamica(risultati):
    indirizzi_corretti = [val for val in risultati.values() if val[0]]
    if indirizzi_corretti:
        centro_lat, centro_long = indirizzi_corretti[0][1], indirizzi_corretti[0][2]
    else:
        centro_lat, centro_long = 0,0
    mappa = folium.Map(location=[centro_lat, centro_long], zoom_start=6)
 
 
    for indirizzo, (valido, lat,log, citta, regione, stato) in risultati.items():
        if valido:
            popup_info = f"Indirizzo : {indirizzo} <br> Città: {citta}  <br> Regione: {regione} <br> Stato: {stato} <br>  coordinate ({lat}, {log})"
            folium.Marker(
                [lat, log],
                popup=folium.Popup(popup_info, max_width=300),
                tooltip=f"{citta}"
            ).add_to(mappa)

    return mappa


In [8]:
map_interactive = mappa_dinamica(output)

save map

In [9]:
map_interactive.save(r"mappa_indirizzi.html")
print("La mappa è stata salvata come 'mappa_indirizzi.html'. Apri il file per visualizzarla.")

La mappa è stata salvata come 'mappa_indirizzi.html'. Apri il file per visualizzarla.


In [10]:
import os
print(os.path.abspath("mappa_indirizzi.html"))

/home/denise/mappa_indirizzi.html
